In [43]:
# Entering rotating axis to find GB plane 

from aimsgb import GBInformation

print(GBInformation([0, 1, 2], 30).__str__())

Grain boundary information for rotation axis: 012
Show the sigma values up to 30 (Note: * means twist GB, Theta is the rotation angle)
|  Sigma  |  Theta  | GB plane   | CSL matrix   |
|---------+---------+------------+--------------|
|    3    | 131.81  | (5 -2 1)   | 5  1  0      |
|         |         | (1 2 -1)   | -2  2  1     |
|         |         | (0 1 2)*   | 1 -1  2      |
|    5    |   180   | (0 2 -1)   | 0 -1  0      |
|         |         | (-1 0 0)   | 2  0  1      |
|         |         | (0 1 2)*   | -1  0  2     |
|    7    |  73.4   | (-3 -2 1)  | -3  5  0     |
|         |         | (5 -6 3)   | -2 -6  1     |
|         |         | (0 1 2)*   | 1  3  2      |
|    9    |  96.38  | (-2 -2 1)  | -2  5  0     |
|         |         | (5 -4 2)   | -2 -4  1     |
|         |         | (0 1 2)*   | 1  2  2      |
|   15    |  48.19  | (5 2 -1)   | 5 -1  0      |
|         |         | (-1 2 -1)  | 2  2  1      |
|         |         | (0 1 2)*   | -1 -1  2     |
|   21    |  58

In [44]:
# Using Target plane to find rotation axis

from aimsgb import GBInformation
from itertools import product
import math

# ===== adjust parameter =====
UVW_VALUES   = range(4)      # u, v, w ∈ {0,1,2,3}
MAX_SIGMA    = 30
TARGET_PLANE = (0, 1, 2)
# ===========================

seen = set()
target_abs = (abs(TARGET_PLANE[0]), abs(TARGET_PLANE[1]), abs(TARGET_PLANE[2]))

print("=== Grain boundary search result ===")
print(f"Target plane (abs) : {target_abs}")
print(f"Sigma <= {MAX_SIGMA}\n")

for u, v, w in product(UVW_VALUES, repeat=3):
    g = math.gcd(math.gcd(u, v), w)
    axis = (u//g, v//g, w//g) if g else None

    if not axis or axis in seen:
        continue
    seen.add(axis)

    try:
        lines = str(GBInformation(list(axis), MAX_SIGMA)).splitlines()
    except Exception:
        continue

    sigma = theta = None

    for line in lines:
        cols = [c.strip() for c in line.split("|") if c.strip()]
        if cols and cols[0].isdigit():
            sigma, theta = cols[0], cols[1]

        if "(" not in line or ")" not in line:
            continue

        inside = line.split("(", 1)[1].split(")", 1)[0]  # e.g. "0 1 2" or "0 -1 2"
        parts = inside.split()
        if len(parts) != 3 or not all(p.lstrip("-").isdigit() for p in parts):
            continue

        h, k, l = map(int, parts)
        if (abs(h), abs(k), abs(l)) != target_abs:
            continue

        gb_type = "TWIST" if "*" in line else "TILT"
        star = "*" if gb_type == "TWIST" else ""
        print(f"{gb_type:5} | axis={axis} | sigma={sigma} | theta={theta} | plane=({h} {k} {l}){star}")


=== Grain boundary search result ===
Target plane (abs) : (0, 1, 2)
Sigma <= 30

TWIST | axis=(0, 1, 2) | sigma=3 | theta=131.81 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=5 | theta=180 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=7 | theta=73.4 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=9 | theta=96.38 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=15 | theta=48.19 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=21 | theta=58.41 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=23 | theta=163.04 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=27 | theta=35.43 | plane=(0 1 2)*
TWIST | axis=(0, 1, 2) | sigma=29 | theta=112.29 | plane=(0 1 2)*
TILT  | axis=(0, 2, 1) | sigma=5 | theta=180 | plane=(0 -1 2)
TILT  | axis=(1, 0, 0) | sigma=5 | theta=53.13 | plane=(0 1 2)
TILT  | axis=(1, 2, 1) | sigma=5 | theta=101.54 | plane=(0 -1 2)
TILT  | axis=(3, 2, 1) | sigma=25 | theta=63.9 | plane=(0 -1 2)


In [46]:
from aimsgb import GrainBoundary, Grain
from pymatgen.core import Structure

base_path = "/Users/emilydai/Library/CloudStorage/OneDrive-個人/文件/敏求計算資源/AI計畫/Random"
save_path = base_path + "/LATP_GB.vasp"

input_structure = Grain.from_file('/Users/emilydai/Library/CloudStorage/OneDrive-個人/文件/Ncku lab/ML_LATP/poscar/LATP_2Al/POSCAR')
#    GrainBoundary(axis, sigma, plane, initial_struct, uc_a=1, uc_b=1)
gb = GrainBoundary([0, 1, 2], 15, [0, 1, 2], input_structure)
structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)

structure = structure.get_sorted_structure()
structure.to(fmt="poscar", filename=save_path)

'Li480 Ti600 Al120 P1080 O4320\n1.0\n  57.5287887761048609    0.0000000000000000    0.0000000000000035\n  11.4129236464627848   23.0807502707791983    0.0000000000000016\n   0.0000000000000000    0.0000000000000000   60.8675216975946540\nLi Ti Al P O\n480 600 120 1080 4320\ndirect\n   0.1012903231662055    0.0294187846155368    0.0877545746285837 Li\n   0.1844736438007875    0.1138109298959069    0.1877545746285837 Li\n   0.1841736184032851    0.4492618871233134    0.3877545746285839 Li\n   0.2679569898328722    0.8627521179488702    0.0877545746285837 Li\n   0.2676569644353696    0.1982030751762769    0.2877545746285837 Li\n   0.2673569390378672    0.5336540324036835    0.4877545746285838 Li\n   0.3511403104674542    0.9471442632292401    0.1877545746285837 Li\n   0.3508402850699516    0.2825952204566466    0.3877545746285839 Li\n   0.4346236564995384    0.6960854512822036    0.0877545746285836 Li\n   0.4343236311020362    0.0315364085096101    0.2877545746285837 Li\n   0.434023605704